<a href="https://colab.research.google.com/github/AvdhutBashte1605/PRODIGY_GA_04/blob/main/TASK4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision matplotlib

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.utils import save_image
import os
import numpy as np
from PIL import Image

In [ ]:
class UNetGenerator(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetGenerator, self).__init__()
        self.encoder = nn.Sequential(
            self._block(in_channels, 64, 4, 2, 1),
            self._block(64, 128, 4, 2, 1),
            self._block(128, 256, 4, 2, 1),
            self._block(256, 512, 4, 2, 1),
            self._block(512, 512, 4, 2, 1),
            self._block(512, 512, 4, 2, 1),
            self._block(512, 512, 4, 2, 1),
        )
        self.decoder = nn.Sequential(
            self._upblock(512, 512, 4, 2, 1),
            self._upblock(1024, 512, 4, 2, 1),
            self._upblock(1024, 512, 4, 2, 1),
            self._upblock(1024, 256, 4, 2, 1),
            self._upblock(512, 128, 4, 2, 1),
            self._upblock(256, 64, 4, 2, 1),
            nn.ConvTranspose2d(128, out_channels, 4, 2, 1),
            nn.Tanh()
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def _upblock(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        enc1 = self.encoder[0](x)
        enc2 = self.encoder[1](enc1)
        enc3 = self.encoder[2](enc2)
        enc4 = self.encoder[3](enc3)
        enc5 = self.encoder[4](enc4)
        enc6 = self.encoder[5](enc5)
        enc7 = self.encoder[6](enc6)
        dec1 = self.decoder[0](enc7)
        dec2 = self.decoder[1](torch.cat([dec1, enc6], dim=1))
        dec3 = self.decoder[2](torch.cat([dec2, enc5], dim=1))
        dec4 = self.decoder[3](torch.cat([dec3, enc4], dim=1))
        dec5 = self.decoder[4](torch.cat([dec4, enc3], dim=1))
        dec6 = self.decoder[5](torch.cat([dec5, enc2], dim=1))
        dec7 = self.decoder[6](torch.cat([dec6, enc1], dim=1))
        return self.decoder[7](dec7)


In [ ]:
class PatchGANDiscriminator(nn.Module):
    def __init__(self, in_channels):
        super(PatchGANDiscriminator, self).__init__()
        self.model = nn.Sequential(
            self._block(in_channels * 2, 64, 4, 2, 1),
            self._block(64, 128, 4, 2, 1),
            self._block(128, 256, 4, 2, 1),
            self._block(256, 512, 4, 1, 1),
            nn.Conv2d(512, 1, 4, 1, 1)
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x, y):
        return self.model(torch.cat([x, y], dim=1))


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = sorted(os.listdir(root_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_paths[idx])
        img = Image.open(img_path).convert("RGB")
        w, h = img.size
        input_image = img.crop((0, 0, w // 2, h))
        target_image = img.crop((w // 2, 0, w, h))
        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)
        return input_image, target_image


In [ ]:
def train_pix2pix(dataloader, generator, discriminator, g_optimizer, d_optimizer, criterion_gan, criterion_l1, device, num_epochs=100):
     # Create results directory if it does not exist
    if not os.path.exists('results'):
        os.makedirs('results')

    for epoch in range(num_epochs):
        for i, (input_image, target_image) in enumerate(dataloader):
            input_image, target_image = input_image.to(device), target_image.to(device)

            # Train Discriminator
            d_optimizer.zero_grad()
            real_output = discriminator(input_image, target_image)
            fake_image = generator(input_image)
            fake_output = discriminator(input_image, fake_image.detach())
            d_loss_real = criterion_gan(real_output, torch.ones_like(real_output))
            d_loss_fake = criterion_gan(fake_output, torch.zeros_like(fake_output))
            d_loss = (d_loss_real + d_loss_fake) / 2
            d_loss.backward()
            d_optimizer.step()

            # Train Generator
            g_optimizer.zero_grad()
            fake_output = discriminator(input_image, fake_image)
            g_loss_gan = criterion_gan(fake_output, torch.ones_like(fake_output))
            g_loss_l1 = criterion_l1(fake_image, target_image)
            g_loss = g_loss_gan + 100 * g_loss_l1
            g_loss.backward()
            g_optimizer.step()

            if i % 50 == 0:
                print(f'Epoch [{epoch}/{num_epochs}], Step [{i}/{len(dataloader)}], d_loss: {d_loss.item()}, g_loss: {g_loss.item()}')

        save_image(fake_image, f'results/fake_image_epoch_{epoch}.png')
        save_image(target_image, f'results/real_image_epoch_{epoch}.png')
        save_image(input_image, f'results/input_image_epoch_{epoch}.png')

    torch.save(generator.state_dict(), 'generator.pth')
    torch.save(discriminator.state_dict(), 'discriminator.pth')


In [ ]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Hyperparameters
    in_channels = 3
    out_channels = 3
    lr = 0.0002
    batch_size = 16
    num_epochs = 200
    image_size = 256

    # Transforms
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Dataset and DataLoader
    dataset = ImageDataset(root_dir="/content/drive/MyDrive/facades/train", transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Models
    generator = UNetGenerator(in_channels, out_channels).to(device)
    discriminator = PatchGANDiscriminator(in_channels).to(device)

    # Optimizers
    g_optimizer = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    d_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    # Loss functions
    criterion_gan = nn.BCEWithLogitsLoss()
    criterion_l1 = nn.L1Loss()

    # Train the Pix2Pix model
    train_pix2pix(dataloader, generator, discriminator, g_optimizer, d_optimizer, criterion_gan, criterion_l1, device, num_epochs)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:952: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv_transpose2d(
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  ret

Epoch [0/200], Step [0/25], d_loss: 0.7063785791397095, g_loss: 86.5526351928711
Epoch [1/200], Step [0/25], d_loss: 0.22985190153121948, g_loss: 48.869224548339844
Epoch [2/200], Step [0/25], d_loss: 0.3069782257080078, g_loss: 38.6231689453125
Epoch [3/200], Step [0/25], d_loss: 0.05645301938056946, g_loss: 44.893489837646484
Epoch [4/200], Step [0/25], d_loss: 0.1416645348072052, g_loss: 37.333412170410156
Epoch [5/200], Step [0/25], d_loss: 0.29096367955207825, g_loss: 36.02899932861328
Epoch [6/200], Step [0/25], d_loss: 0.13830724358558655, g_loss: 39.27992248535156
Epoch [7/200], Step [0/25], d_loss: 0.1458873450756073, g_loss: 37.067039489746094
Epoch [8/200], Step [0/25], d_loss: 0.4076288342475891, g_loss: 35.10472869873047
Epoch [9/200], Step [0/25], d_loss: 0.21816307306289673, g_loss: 33.92790985107422
Epoch [10/200], Step [0/25], d_loss: 0.1434810906648636, g_loss: 35.14015579223633
Epoch [11/200], Step [0/25], d_loss: 0.1918073445558548, g_loss: 35.85015869140625
Epoch [

In [ ]:
import torch
from torchvision.utils import save_image
from PIL import Image

# Load the generator
generator = UNetGenerator(in_channels=3, out_channels=3)
generator.load_state_dict(torch.load('generator.pth'))
generator.eval()

# Transform for input images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

def generate_image(input_image_path, output_image_path):
    input_image = Image.open(input_image_path).convert("RGB")
    input_image = transform(input_image).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        fake_image = generator(input_image)

    save_image(fake_image, output_image_path)

# Generate an image
generate_image('/content/drive/MyDrive/content.jpg', '/content/drive/MyDrive/generated_image.png')
